In [1]:
import numpy as np
import tensorflow as tf
#https://gist.github.com/danijar/d11c77c5565482e965d1919291044470

In [2]:
data = open('dinos.txt', 'r').read()
data = data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 19909 total characters and 27 unique characters in your data.


In [3]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [4]:
def word_to_one_vector(word, char_to_ix):
    one_hot_vectors = np.zeros((len(word), len(char_to_ix)))
    one_hot_vectors[ np.arange(len(word)), [char_to_ix[ch] for ch in word] ] = 1
    return one_hot_vectors

In [5]:
with open("dinos.txt") as f:
    examples = f.readlines()
examples = [x.lower().strip() for x in examples]

In [6]:
n_neurons = 150

In [7]:
X = tf.placeholder(tf.float32, [None, None, vocab_size])
y = tf.placeholder(tf.int32, [None, None]) #Shape => Batch_Size x Steps
seq_len = tf.placeholder(tf.int32)

In [8]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units =  n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype = tf.float32, sequence_length = seq_len)

In [9]:
stacked_outputs = tf.reshape(outputs, [-1, n_neurons])
stacked_outputs_dense = tf.layers.dense(stacked_outputs, vocab_size)
outputs_2 = tf.reshape(stacked_outputs_dense, [-1, seq_len, vocab_size])

In [10]:
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = outputs_2)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.0001)
training_op = optimizer.minimize(loss)

In [11]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(50000):
        for iteration, example  in enumerate(examples):
            x_ = np.vstack((np.zeros(len(char_to_ix)),word_to_one_vector(example, char_to_ix)))
            x_ = x_.reshape(1, x_.shape[0], x_.shape[1])
            y_ = np.append(np.asarray([(char_to_ix[x]) for x in example]),0)
            y_ = y_.reshape(1, y_.shape[0])
            sess.run(training_op, feed_dict = {X: x_, y: y_, seq_len: x_.shape[1]})
            #out = sess.run(outputs_2, feed_dict = {X: x_, y: y_, seq_len: x_.shape[1]})
            #print(out.shape, y_)
            if iteration % 1000 == 0:
                print(loss.eval(feed_dict = {X: x_, y: y_, seq_len: x_.shape[1]}))
                #out = sess.run(outputs_2, feed_dict = {X: np.zeros((10,len(char_to_ix))).reshape(1,10,len(char_to_ix)), seq_len: 10})
                #indexs = tf.argmax(tf.nn.softmax(out), 1).eval()[0]
                #word = ''.join([ ix_to_char[x] for x in indexs])
                #print(word)
            


3.4128556
3.3760765
3.2767794
3.323585
3.092558
3.2371554
2.8374934
3.1709425
2.6711676
3.153935
2.590195
3.1550362
2.5439563
3.159086
2.511801
3.1624403
2.4859173
3.1644309
2.4628508
3.164948
2.440867
3.1639836
2.4190328
3.161592
2.3968647
3.1579154
2.3741896
3.1531978
2.3510711
3.1477582
2.327732
3.1419206
2.304459
3.1359317
2.2815194
3.1299195
2.2591016
3.1239045
2.237302
3.1178396
2.2161376
3.1116552
2.1955686
3.1052883
2.175525
3.098696
2.1559215
3.091866
2.1366742
3.0848107
2.1177068
3.0775597
2.098957
3.0701585
2.08038
3.0626595
2.0619493
3.0551183
2.0436566
3.047588
2.0255136
3.040117
2.007548
3.0327485
1.9898013
3.0255177
1.9723244
3.018454
1.9551765
3.01158
1.9384168
3.0049093
1.9221056
2.9984524
1.9062947
2.99221
1.8910306
2.986176
1.8763477
2.9803405
1.8622692
2.9746878
1.8488063
2.9691978
1.8359587
2.9638505
1.8237158
2.9586272
1.812059
2.9535089
1.8009626
2.9484813
1.7903974
2.9435344
1.7803307
2.9386618
1.7707288
2.9338593
1.7615601
2.9291267
1.7527914
2.9244666
1.744392